In [1]:
import cv2

In [2]:
# Загрузка каскадов для обнаружения рук
hand_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_hand.xml')

[ERROR:0@0.515] global persistence.cpp:519 open Can't open file: '/home/redalexdad/anaconda3/envs/dl_science/lib/python3.9/site-packages/cv2/data/haarcascade_hand.xml' in read mode


In [3]:
# Загрузка изображения часов
clock_image = cv2.imread('clock.png', cv2.IMREAD_UNCHANGED)

[ WARN:0@2.956] global loadsave.cpp:248 findDecoder imread_('clock.png'): can't open/read file: check file path/integrity


In [4]:
# Функция для наложения часов на руки
def overlay_clock(hand_roi, clock_image):
    # Изменяем размер изображения часов, чтобы оно соответствовало размеру руки
    clock_resized = cv2.resize(clock_image, (hand_roi.shape[1], hand_roi.shape[0]))
    
    # Извлекаем альфа-канал из изображения часов
    clock_alpha = clock_resized[:, :, 3] / 255.0
    
    # Извлекаем каналы BGR из изображения часов
    clock_bgr = clock_resized[:, :, 0:3]
    
    # Накладываем часы на руку с учетом альфа-канала
    hand_roi = hand_roi.astype(float)
    overlay = (clock_bgr.astype(float) * clock_alpha[:,:,None]) + (hand_roi * (1 - clock_alpha[:,:,None]))
    overlay = overlay.astype('uint8')
    
    return overlay

In [5]:
# Захватываем видеопоток с камеры
cap = cv2.VideoCapture(0)

In [6]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Преобразовываем изображение в оттенки серого
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Обнаружение рук
    hands = hand_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Наложение часов на руки
    for (x, y, w, h) in hands:
        hand_roi = frame[y:y+h, x:x+w]
        frame[y:y+h, x:x+w] = overlay_clock(hand_roi, clock_image)
    
    # Отображение результата
    cv2.imshow('Clock Overlay', frame)
    
    # Выход из цикла при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

error: OpenCV(4.9.0) /io/opencv/modules/objdetect/src/cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'detectMultiScale'


In [7]:
# Освобождаем ресурсы
cap.release()
cv2.destroyAllWindows()